In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
import time
import seaborn as sn
from sklearn.metrics import confusion_matrix

ImportError: DLL load failed while importing _arpack: The specified procedure could not be found.

In [2]:
pip install tensorflow-directml-plugin

  Using cached tensorflow_directml_plugin-0.4.0.dev230202-cp39-cp39-win_amd64.whl (9.0 MB)
  Using cached tensorflow_cpu-2.10.0-cp39-cp39-win_amd64.whl (2.0 kB)
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: grpcio>=1.8.6 in c:\users\shalt\anaconda3\lib\site-packages (from tensorflow-directml-plugin) (1.42.0)
  Using cached tensorflow_intel-2.10.0-cp39-cp39-win_amd64.whl (262.6 MB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached libclang-15.0.6.1-py2.py3-none-win_amd64.whl (23.2 MB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached termcolor-2.2.0-py3-none-any.whl (6.6 kB)

  Using cached flatbuffers-23.1.21-py2.py3-none-any.whl (26 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached 

In [31]:
data = pd.read_csv('hisotryWithRollingAverages.csv')

In [32]:
data.head()

,Unnamed: 0,Date time,Minimum Temperature,Maximum Temperature,Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,...,Weather Type,Conditions,temp_c,Month,Year,Temperature_Lag1,Temperature_Lag2,Precipitation_Lag1,Precipitation_Lag2,temp_7d_rolling
0,0,2023-02-01 00:00:00,50.9,50.9,50.9,49.1,93.53,NaN,7.0,NaN,...,Thunderstorm,Overcast,10.500000,2,2023,NaN,NaN,NaN,NaN,NaN
1,1,2023-02-01 01:00:00,44.0,44.0,44.0,39.5,84.07,NaN,13.0,NaN,...,Thunderstorm,Overcast,6.666667,2,2023,50.9,NaN,0.0,NaN,NaN
2,2,2023-02-01 02:00:00,44.7,44.7,44.7,39.9,82.97,NaN,10.1,NaN,...,"Mist, Rain, Thunderstorm",Overcast,7.055556,2,2023,44.0,50.9,0.0,0.0,NaN
3,3,2023-02-01 03:00:00,44.7,44.7,44.7,39.9,82.95,NaN,14.3,NaN,...,"Thunderstorm, Light Rain",Overcast,7.055556,2,2023,44.7,44.0,0.0,0.0,NaN
4,4,2023-02-01 04:00:00,44.7,44.7,44.7,39.9,82.97,NaN,9.8,NaN,...,"Rain, Thunderstorm",Overcast,7.055556,2,2023,44.7,44.7,0.0,0.0,NaN


In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439918 entries, 0 to 439917
Data columns (total 27 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           439918 non-null  int64  
 1   Date time            439918 non-null  object 
 2   Minimum Temperature  421821 non-null  float64
 3   Maximum Temperature  421821 non-null  float64
 4   Temperature          421821 non-null  float64
 5   Dew Point            421543 non-null  float64
 6   Relative Humidity    421543 non-null  float64
 7   Heat Index           60366 non-null   float64
 8   Wind Speed           421543 non-null  float64
 9   Wind Gust            1026 non-null    float64
 10  Wind Direction       397295 non-null  float64
 11  Wind Chill           40460 non-null   float64
 12  Precipitation        439918 non-null  float64
 13  Snow Depth           196268 non-null  float64
 14  Visibility           421334 non-null  float64
 15  Cloud Cover      

In [22]:
train_labels_onehot = pd.get_dummies(data.Conditions).values
train_labels_onehot

array([[0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=uint8)

In [35]:
x_vectors = data.drop('Conditions', axis=1)

In [37]:
Weather_Type.getdummies()

AttributeError: 'DataFrame' object has no attribute 'getdummies'

In [26]:
df_int = data.select_dtypes(include=['float']).astype(int, errors='ignore')
print(df_int.info())
data = pd.concat([df_int, data.select_dtypes(exclude=['float'])], axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439918 entries, 0 to 439917
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Minimum Temperature  421821 non-null  float64
 1   Maximum Temperature  421821 non-null  float64
 2   Temperature          421821 non-null  float64
 3   Dew Point            421543 non-null  float64
 4   Relative Humidity    421543 non-null  float64
 5   Heat Index           60366 non-null   float64
 6   Wind Speed           421543 non-null  float64
 7   Wind Gust            1026 non-null    float64
 8   Wind Direction       397295 non-null  float64
 9   Wind Chill           40460 non-null   float64
 10  Precipitation        439918 non-null  float64
 11  Snow Depth           196268 non-null  float64
 12  Visibility           421334 non-null  float64
 13  Cloud Cover          358298 non-null  float64
 14  Sea Level Pressure   185891 non-null  float64
 15  temp_c           

In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439918 entries, 0 to 439917
Data columns (total 27 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Minimum Temperature  421821 non-null  float64
 1   Maximum Temperature  421821 non-null  float64
 2   Temperature          421821 non-null  float64
 3   Dew Point            421543 non-null  float64
 4   Relative Humidity    421543 non-null  float64
 5   Heat Index           60366 non-null   float64
 6   Wind Speed           421543 non-null  float64
 7   Wind Gust            1026 non-null    float64
 8   Wind Direction       397295 non-null  float64
 9   Wind Chill           40460 non-null   float64
 10  Precipitation        439918 non-null  float64
 11  Snow Depth           196268 non-null  float64
 12  Visibility           421334 non-null  float64
 13  Cloud Cover          358298 non-null  float64
 14  Sea Level Pressure   185891 non-null  float64
 15  temp_c           

In [ ]:
def getTeamsStats(df, numOfRecords = 10):
    """
    gets stats for each team from the past 'numOfRecords' games, split into home / away statistics. relevant stats are the
    aggregated fields such as total / avg fields
    """
    lastAwayGames = pd.DataFrame(df[df['is_home_team'] == False].sort_values(['game_date'], ascending=False).groupby(['team_name']).head(numOfRecords))
    lastHomeGames = pd.DataFrame(df[df['is_home_team'] == True].sort_values(['game_date'], ascending=False).groupby(['team_name']).head(numOfRecords))
    fieldsToCalculate = [col for col in list(df.columns) if col.startswith('total') or col.startswith('avg')] + ['game_won']
    teamsStats = {teamName: {'home': dict(), 'away': dict()} for teamName in df.team_name.unique()}
    for colName in fieldsToCalculate:
        for teamName, mean in dict(lastHomeGames.groupby(['team_name'])[colName].mean()).items():
            teamsStats[teamName]['home'][f'{colName}_mean_home'] = mean
        for teamName, mean in dict(lastAwayGames.groupby(['team_name'])[colName].mean()).items():
            teamsStats[teamName]['away'][f'{colName}_mean_away'] = mean
    return teamsStats

In [ ]:
def trainModels(numberOfLayersToTest, epochsToTest, batchSizesToTest, trainData, trainLabels):
    fitModels = dict()
    histories = dict()
    for numberOfLayers in numberOfLayersToTest:
        for epochs in epochsToTest:
            for batchSize in batchSizesToTest:
                model = generateModelPerLayerNumber(numberOfLayers)
                history, fitModel = dynamicTrainModel(trainData, trainLabels, epochs, batchSize, model)
                key = f'numberOfLayers is {numberOfLayers}. epochs is {epochs}. batchSize is {batchSize}'
                testModel(history, f'accuracy vs loss, {numberOfLayers} layers, {epochs} epochs, {batchSize} batchSize')
                fitModels[key] = fitModel
                histories[key] = history
    return histories, fitModels

In [ ]:
numberOfLayersToTest = [3, 7, 10, 15, 20, 50]
epochsToTest = [10, 20, 30]
batchSizesToTest = [50, 100, 150]
histories, models = trainModels(numberOfLayersToTest, epochsToTest, batchSizesToTest, splitTrainImages, splitTrainLabels)

In [ ]:
def dynamicTrainModel(trainData, trainLabels, epochs, batchSize, model):
    imagesDf = model.fit(
        trainData, trainLabels, 
        validation_data = None,
        epochs = epochs,
        batch_size = batchSize,
        verbose = 1
    )
    return imagesDf.history, model

In [12]:
def generateModelPerLayerNumber(numberOfLayers):
    """
    generates a model with numOfLayers layers, with the first layer with 512 dimensions, and the last layer with 10, and the 
    rest are equally spread in between linearly. for example 5 layers will be 512, 387, 262, 137, 10
    """
    model = Sequential()
    diff = 502 // (numberOfLayers - 1) # difference between each dimension
    
    model.add(Dense(512, input_dim=28*28, activation='relu'))
    
    for i in range (1, numberOfLayers - 1):
        model.add(Dense(512 - diff * i, 'relu'))
    
    model.add(Dense(10, activation='sigmoid'))
    model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [29]:
model = generateModelPerLayerNumber(3)

In [30]:
model.fit(data.drop('Conditions', axis=1), data.Conditions, validation_split=0.2, epochs=10, verbose=1)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [15]:
data.Conditions.unique()

array(['Overcast', 'Rain, Overcast', 'Partially cloudy',
       'Rain, Partially cloudy', 'Clear', 'Rain', nan], dtype=object)